# Evaluate song not in dataset

## Load libraries

In [1]:
import librosa
import numpy as np
import tensorflow as tf
import peakutils
import os

## Load custom modules

In [2]:
from nnssa.constants import *
from nnssa.sub_divisions import *
from nnssa.spectrograms import extract_melspecs
from nnssa.evaluate import post_processing

## Set constants

In [3]:
ROOT = os.path.join(os.getcwd(), '..', '..') 

## Load song

In [4]:
mp3, _ = librosa.load(os.path.join(ROOT, 'data', '07_Test', 'hot_cold.mp3'), sr=SR)
mp3.shape

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(4855998,)

## Generate Mel spectrograms

In [5]:
melspec = extract_melspecs(mp3)
melspec.shape

(80, 4743)

## Get beat times

In [6]:
tempo, beats = librosa.beat.beat_track(mp3, sr=SR)
times = librosa.frames_to_time(beats, sr=SR, hop_length=HOP_SIZE)

## Create beat-wise spectrograms

In [7]:
sync = librosa.util.sync(melspec, beats) 

## Create sub-divisions

In [8]:
subs = create_spec_windows(sync, 16, 4)
subs.shape

(64, 80, 4, 33)

## Normalize

In [9]:
subs = normalize(subs)
subs.shape

(64, 80, 4, 33)

## Load models

In [10]:
M = 5
models = [None] * 5
for m in range(M):
    models[m] = tf.keras.models.load_model(os.path.join(ROOT, 'models', f'model_{m}.h5'))

## Make predictions

In [11]:
votes = np.zeros((M, subs.shape[0]))
for m in range(M):
    m_preds = models[m].predict([subs], batch_size=1, verbose=1)
    votes[m] = m_preds.flatten()

64/64 [==============================] - 0s 769us/step


In [12]:
preds = votes.sum(axis=0)/5

## Get peaks

In [13]:
p = post_processing(preds.squeeze())
peaks = peakutils.indexes(p, min_dist=4, thres=0.01)

## Split peaks to bars

In [14]:
peaks = [(p * 4) for p in peaks if (p * 4) < len(times)]

## Print boundaries

In [15]:
boundaries = times[peaks]

In [16]:
boundaries

array([  3.85451247,  32.92589569,  65.66603175,  98.4061678 ,
       131.14630385, 171.13106576, 225.69795918])